In [11]:
%matplotlib inline 
from IPython.core.display import display

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np

from sympy import *
init_printing()
#from math import *
from interval import interval
from interval import imath as im
import sys
from pprint import pprint

<img  src="スクリーンショット 2016-12-29 16.59.29.png"/>

<img  src="スクリーンショット 2016-12-29 19.29.13.png"/>

In [12]:
class ivmat(list):
    class DimentionNotMatchError(Exception):
        pass
    
    class NotListError(Exception):
        pass
    
    class SizeNotMatchError(Exception):
        pass
    
    class NotMidpointError(Exception):
        pass
 
    def _get_shape(self, X):
        if isinstance(X, list):            
            for index, x in enumerate(X):
                if index==0 and isinstance(x, list):
                    col_num = len(x)
                elif isinstance(x, list):
                    if not col_num == len(x):
                        raise DimentionNotMatchError()
                else:
                    raise NotListError()
            return (len(X), col_num)
        else:
            raise NotListError()
    
    def __init__(self, args):
        self.shape = self._get_shape(args)
        self.size = reduce(lambda x,y: x*y, self.shape)
        #self.midpoint = self._get_midpoint()
        super(ivmat, self).__init__(args)
    
    def __pos__(self): # +x
        return self
    
    def __neg__(self): # -x
        return ivmat(map(lambda x_row: 
                         [-x for x in x_row], 
                         self))
    
    def __add__(self, other): # x + y
        return ivmat(map(lambda (x_row,y_row): 
                         [x+y for x,y in zip(x_row, y_row)], 
                         zip(self, other)))
        
    def __radd__(self, other): # y + x
        return ivmat(map(lambda (x_row,y_row): 
                         [x+y for x,y in zip(x_row, y_row)], 
                         zip(self, other)))
    
    def __sub__(self, other): # x - y
        return ivmat(map(lambda (x_row,y_row): 
                         [x-y for x,y in zip(x_row, y_row)], 
                         zip(self, other)))
    
    def __mul__(self, other): # x * y
        return ivmat(map(lambda (x_row,y_row): 
                         [x*y for x,y in zip(x_row, y_row)], 
                         zip(self, other)))
        
    def __rmul__(self, other): # y * x
        return ivmat(map(lambda (x_row,y_row): 
                         [x*y for x,y in zip(x_row, y_row)], 
                         zip(self, other)))
    
    def __and__(self, other):
        return ivmat(map(lambda (x_row,y_row): 
                         [x&y for x,y in zip(x_row, y_row)], 
                         zip(self, other)))
    
        
    @classmethod
    def dot(self, x, y): # matrix operation
        if not len(x[0]) == len(y):
            raise DimentionNotMatchError('dot dimention unmatch')
        mat = ivmat([[None for col in range(len(y[0]))] for row in range(len(x))])
        for i in range(len(x)):
            for j in range(len(y[0])):
                sum = 0
                for k in range(len(x[0])):
                    sum += x[i][k] * y[k][j]
                mat[i][j] = sum
        return mat
    
    @property
    def midpoint(self):
        mat = ivmat([[None for col in range(self.shape[1])] for row in range(self.shape[0])])
        for i in range(mat.shape[0]):
            for j in range(mat.shape[1]):
                mat[i][j] = self[i][j].midpoint
        return mat
    
    def _flatten(self): # 1-D
        return reduce(lambda x,y: x+y, self)
    
    def reshape(self, row_num, col_num):
        if not row_num * col_num == self.size:
            raise SizeNotMatchError('reshape method cannnot change elements size')    
        mat = ivmat([[None for col in range(col_num)] for row in range(row_num)])
        for i, x in enumerate(self._flatten()):
            row_index = i // col_num
            col_index = i % col_num
            mat[row_index][col_index] = x
        return mat
    
    def is_midpoint(self):
        for i in range(self.shape[0]):
            for j in range(self.shape[1]):
                iv = self[i][j]
                if not iv[0][0] == iv[0][1]:
                    return False
        return True
    
    def to_scalar(self):
        mat = ivmat([[None for col in range(self.shape[1])] for row in range(self.shape[0])])        
        for i in range(mat.shape[0]):
            for j in range(mat.shape[1]):
                iv = self[i][j]
                if not self.is_midpoint(): 
                    raise NotMidpointError()
                mat[i][j] = iv[0][0]
        return mat
    
    def to_interval(self):
        mat = ivmat([[None for col in range(self.shape[1])] for row in range(self.shape[0])])        
        for i in range(mat.shape[0]):
            for j in range(mat.shape[1]):
                mat[i][j] = interval.cast(self[i][j])
        return mat
    
    def get_pinv(self):
        np_pinv =  np.around(np.linalg.pinv(np.array(self)), decimals=2)
        return ivmat(np_pinv.tolist()).to_interval()
    
    @classmethod
    def eye(self, n):
        """
        Returns:
            ans = ivmat(
                [1,0,...,0],
                [0,1,...,0],
                        :
                [0,...,0,1]
            )
            ans.shape == (n,n)
        """
        return ivmat(np.eye(n).tolist())
        
    
a = ivmat([[interval[-2,1]], [interval[3,4]], [interval[2,3]]])
b = ivmat([[interval[-5,-4]], [interval[-3,1]], [interval[-1,2]]])
c = ivmat([[interval[2,3], interval[5,7], interval[0,1]]])
d = ivmat([[interval[1,2], interval[-2,1], interval[4,6]]])

from itertools import chain

print b
print a+b
print a-b
print a*b
print
print a
print(a.midpoint)

[[interval([-5.0, -4.0])], [interval([-3.0, 1.0])], [interval([-1.0, 2.0])]]
[[interval([-7.0, -3.0])], [interval([-0.0, 5.0])], [interval([1.0, 5.0])]]
[[interval([2.0, 6.0])], [interval([2.0, 7.0])], [interval([-0.0, 4.0])]]
[[interval([-5.0, 10.0])], [interval([-12.0, 4.0])], [interval([-3.0, 6.0])]]

[[interval([-2.0, 1.0])], [interval([3.0, 4.0])], [interval([2.0, 3.0])]]
[[interval([-0.5])], [interval([3.5])], [interval([2.5])]]


In [13]:
x_1, x_2, x_3 = var("x_1 x_2 x_3")
f_1 = symbols("f_1", cls=Function)
f_2 = symbols("f_2", cls=Function)
f_1 = x_1**2 + x_2**2 - 1
f_2 = x_1 - x_2**2

display(f_1)
display(f_2)

class fmat(list):
    def apply_args(self,X_mat):
        """
        Params:
            X_mat: ivmat([[x_1], [x_2]])
        Returns:
            ivmat
        """
        X = map(lambda x: x[0], X_mat)        
        ans = []
        for f_row in self:
            tmp = []
            for f in f_row:
                tmp.append(f(*X))
            ans.append(tmp)
        return ivmat(ans)
            

f_strs = ('f_1', 'f_2')
x_strs = ('x_1', 'x_2')
grad_strs = [[None for i in range(len(x_strs))] for j in range(len(x_strs))]

f_grad = fmat()
for i,f in enumerate(['f_1', 'f_2']):
    tmp = []
    for j,x in enumerate(['x_1', 'x_2']):
        grad_strs[i][j] = str(eval(f).diff(eval(x)))
        print grad_strs[i][j]
        workspace = dict([('i', i), ('j', j)])
        grad = lambda x_1, x_2: interval(eval(grad_strs[i][j]))
        tmp.append(grad)
    f_grad.append(tmp)
display(f_grad)

print '---なぜ同じ結果に？---'*2  
print "多分、eval(gtad_str)は実行時に評価されるので最後のgrad_strが使われる"
print f_grad[0][0](interval[0.5,0.8], interval[0.6,0.9])
print f_grad[0][1](interval[0.5,0.8], interval[0.6,0.9])
print f_grad[1][0](interval[0.5,0.8], interval[0.6,0.9])
print f_grad[1][1](interval[0.5,0.8], interval[0.6,0.9])
print '------'*10

"""
このようにそれぞれを作るとうまくいく
"""
f_1_str = str(f_1)
f_2_str = str(f_2)
f = fmat([[lambda x_1,x_2: eval(f_1_str)], [lambda x_1,x_2: eval(f_2_str)]])


f1x1_str = str(f_1.diff(x_1)) 
f1x1 = lambda x_1, x_2: interval.cast(eval(f1x1_str))
f1x2_str = str(f_1.diff(x_2)) 
f1x2 = lambda x_1, x_2: interval.cast(eval(f1x2_str))
f2x1_str = str(f_2.diff(x_1)) 
f2x1 = lambda x_1, x_2: interval.cast(eval(f2x1_str))
f2x2_str = str(f_2.diff(x_2)) 
f2x2 = lambda x_1, x_2: interval.cast(eval(f2x2_str))

f_grad = fmat([[f1x1, f1x2], [f2x1, f2x2]])
X = ivmat([[interval[0.5,0.8]], [interval[0.6,0.9]]])
print "別々に作るとうまくいく"
print f_grad.apply_args(X)



2*x_1
2*x_2
1
-2*x_2


[[<function __main__.<lambda>>, <function __main__.<lambda>>],
 [<function __main__.<lambda>>, <function __main__.<lambda>>]]

---なぜ同じ結果に？------なぜ同じ結果に？---
多分、eval(gtad_str)は実行時に評価されるので最後のgrad_strが使われる
interval([-1.8, -1.2])
interval([-1.8, -1.2])
interval([-1.8, -1.2])
interval([-1.8, -1.2])
------------------------------------------------------------
別々に作るとうまくいく
[[interval([1.0, 1.6]), interval([1.2, 1.8])], [interval([1.0]), interval([-1.8, -1.2])]]


In [14]:
X = ivmat([[interval[0.5,0.8]], [interval[0.6,0.9]]])
y = X.midpoint
print y
print f_grad.apply_args(X).midpoint
res =  f_grad.apply_args(X).midpoint.to_scalar()
print res.get_pinv()

Y = ivmat([[0.43,0.43], [0.29,-0.37]])


[[interval([0.65])], [interval([0.75])]]
[[interval([1.3]), interval([1.5])], [interval([1.0]), interval([-1.5])]]
[[interval([0.43]), interval([0.43])], [interval([0.29]), interval([-0.38])]]


In [15]:
y = X.midpoint.to_scalar()
print y
Y = f_grad.apply_args(X).midpoint.to_scalar().get_pinv()
#Y = ivmat([[0.43,0.43], [0.29,-0.37]])
print Y
print f.apply_args(y)
Z = X - y
print Z
I_YF_gradX = ivmat.eye(2) - ivmat.dot(Y, f_grad.apply_args(X))
KX = y - ivmat.dot(Y, f.apply_args(y)) + ivmat.dot(I_YF_gradX, Z)
print '---- KX ----'+'-----'*10

pprint(KX)

pprint(KX & X)

[[0.65], [0.75]]
[[interval([0.43]), interval([0.43])], [interval([0.29]), interval([-0.38])]]
[[-0.014999999999999902], [0.08750000000000002]]
[[interval([-0.15000000000000002, 0.15000000000000002])], [interval([-0.15000000000000002, 0.15000000000000002])]]
---- KX ------------------------------------------------------
[[interval([0.5591249999999999, 0.6785250000000002])],
 [interval([0.7431999999999999, 0.8320000000000002])]]
[[interval([0.5591249999999999, 0.6785250000000002])],
 [interval([0.7431999999999999, 0.8320000000000002])]]


In [18]:
class Krawczyk():
    def __init__(self, f,f_grad, X):
        self.f = f
        self.f_grad = f_grad
        self.X = X
        self.y = self.X.midpoint.to_scalar()
        self.Y = self.f_grad.apply_args(self.X).midpoint.to_scalar().get_pinv()
        self.Z = self.X-self.y
        self.dim = len(self.f)
        
    def run(self, iter_num=10):
        # initialize
        X = self.X
        y = X.midpoint
        Y = self.Y
        Z = self.Z
        pprint(X)
        
        for i in range(iter_num):
            left = y - ivmat.dot(Y, self.f.apply_args(y))
            right = ivmat.dot(ivmat.eye(self.dim) - ivmat.dot(Y, self.f_grad.apply_args(X)), Z)
            KX = left + right
            X = KX & X
            print '---------', i,'------------------'
            #pprint(KX)
            pprint(X)        
            #update
            y = X.midpoint
            Y =  self.f_grad.apply_args(X).midpoint.to_scalar().get_pinv()
            Z = X - y
    

In [19]:

#X = ivmat([[interval[0,1]], [interval[0,1]]])
X = ivmat([[interval[0.5,0.8]], [interval[0.6,0.9]]])
kra = Krawczyk(f,f_grad,X)
kra.run(10)



[[interval([0.5, 0.8])], [interval([0.6, 0.9])]]
--------- 0 ------------------
[[interval([0.5591249999999999, 0.6785250000000002])],
 [interval([0.7431999999999999, 0.8320000000000002])]]
--------- 1 ------------------
[[interval([0.6108581824687497, 0.6251993749687503])],
 [interval([0.7811419024249998, 0.7911914068250002])]]
--------- 2 ------------------
[[interval([0.6178976554300505, 0.6181703869684437])],
 [interval([0.7860155185469165, 0.7862874847206657])]]
--------- 3 ------------------
[[interval([0.6180330888839716, 0.6180348882114599])],
 [interval([0.7861495284671665, 0.7861532296419088])]]
--------- 4 ------------------
[[interval([0.6180339831388258, 0.6180339943634833])],
 [interval([0.7861513567678791, 0.7861513987699036])]]
--------- 5 ------------------
[[interval([0.6180339887149178, 0.6180339887848562])],
 [interval([0.7861513775371982, 0.7861513779778749])]]
--------- 6 ------------------
[[interval([0.6180339887496769, 0.618033988750113])],
 [interval([0.786151

In [20]:
x_1, x_2, x_3 = var("x_1 x_2 x_3")
f_x_1 = symbols("f_x_1", cls=Function)
f_x_2 = symbols("f_x_2", cls=Function)
nn = exp(-4*(x_1-1)**2 - 6*(x_2-0.5)**2) + exp(-7*(x_1+1.5)**2 - 9*(x_2+1)**2)
display(nn)
f = lambda x_1,x_2,exp: eval(str(nn)) 
grad_f_x_1 = str(nn.diff(x_1))
f_x_1 = lambda x_1,x_2,exp=im.exp: eval(grad_f_x_1)
grad_f_x_2 = str(nn.diff(x_2))
f_x_2 = lambda x_1,x_2,exp=im.exp: eval(grad_f_x_2)

f = fmat([[f_x_1], [f_x_2]])
hes = hessian(nn, (x_1,x_2))
func_eval = lambda func: (lambda x_1, x_2, exp=im.exp: eval(str(func)))
f_grad = fmat([[None for i in range(hes.shape[1])] for j in range(hes.shape[0])])
for i in range(2):
    for j in range(2):
        f_grad[i][j] = func_eval(hes[i,j])

x_1= interval[-1.6, -1.45]
x_2 = interval[-0.9,-1.2]
print x_1
X = ivmat([[x_1],[x_2]])
#print f_grad.apply_args(args)




interval([-1.6, -1.45])


In [21]:
kra = Krawczyk(f,f_grad,X)
kra.run(10)

[[interval([-1.6, -1.45])], [interval([-1.2, -0.9])]]
--------- 0 ------------------
[[interval([-1.591533005993967, -1.45])],
 [interval([-1.1615313623148635, -0.9])]]
--------- 1 ------------------
[[interval([-1.560142544889375, -1.45])],
 [interval([-1.0873122646913087, -0.9])]]
--------- 2 ------------------
[[interval([-1.5173586386949545, -1.481430413752573])],
 [interval([-1.0272429930490141, -0.9737645977927148])]]
--------- 3 ------------------
[[interval([-1.5008301403144149, -1.4991456349783487])],
 [interval([-1.0022362436336112, -0.9976831543773185])]]
--------- 4 ------------------
[[interval([-1.5000167348233018, -1.4999827806821817])],
 [interval([-1.000185372856922, -0.9998210753008416])]]
--------- 5 ------------------
[[interval([-1.5000003347119717, -1.499999655598138])],
 [interval([-1.0000143139767526, -0.9999851701706269])]]
--------- 6 ------------------
[[interval([-1.5000000066942416, -1.4999999931119605])],
 [interval([-1.0000011863863503, -0.999998854881859